In [1]:
!pip install langchain faiss-cpu sentence-transformers transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 36.4 MB/s eta 0:00:00


In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [15]:
!pip install fastapi uvicorn pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.4 MB/s eta 0:00:00


In [28]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, Request
from pydantic import BaseModel
from pyngrok import ngrok
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from transformers import pipeline

In [17]:
documents = [
    {"text": "Chris Bumstead, also known as Cbum, is a professional bodybuilder and a four-time Mr. Olympia Classic Physique champion. He is renowned for his aesthetic physique and dedication to the sport of bodybuilding."},
    {"text": "The Mr. Olympia competition is one of the most prestigious bodybuilding contests in the world, held annually by the International Federation of Bodybuilding & Fitness (IFBB)."},
    {"text": "Bodybuilding is a sport that involves rigorous training and strict diets to develop muscle mass and achieve an ideal physique. Competitors are judged based on muscularity, symmetry, and conditioning."},
    {"text": "Chris Bumstead’s training focuses on hypertrophy, emphasizing both heavy compound lifts and high-volume accessory work. He often shares his workout routines and diets with his fans."},
    {"text": "Big Ramy, whose real name is Mamdouh Elssbiay, is a two-time Mr. Olympia champion known for his massive size and incredible stage presence. He is one of the most famous bodybuilders in the open category."},
    {"text": "The Classic Physique category in bodybuilding was introduced to bring back the aesthetics and proportions of the golden era of bodybuilding, emphasizing symmetry and elegance."},
    {"text": "Arnold Schwarzenegger, a seven-time Mr. Olympia champion, is one of the most iconic bodybuilders in history. He helped popularize the sport worldwide."},
    {"text": "Chris Bumstead follows a disciplined nutrition plan that includes lean proteins, complex carbohydrates, and healthy fats to fuel his intense workouts and maintain his physique."},
    {"text": "The Mr. Olympia competition includes several categories, such as Men’s Open, Classic Physique, Men’s Physique, Women’s Bikini, and Women’s Bodybuilding."},
    {"text": "Big Ramy is celebrated for his massive quads and overall size, which set him apart from other competitors in the Men’s Open category."},
    {"text": "Bodybuilding originated in the late 19th century and has since evolved into a globally recognized sport with millions of enthusiasts and followers."},
    {"text": "Chris Bumstead’s journey to the top of the Classic Physique division has inspired many young bodybuilders to pursue the sport with a focus on aesthetics and proportions."},
    {"text": "The judging criteria for Mr. Olympia vary by category but typically include muscle size, symmetry, definition, and stage presentation."},
    {"text": "Chris Bumstead’s rivalry with other top Classic Physique athletes has been one of the highlights of recent Mr. Olympia competitions, showcasing the growing popularity of the division."},
    {"text": "Big Ramy made history as the first Egyptian bodybuilder to win the Mr. Olympia title, bringing pride to his country and fans around the world."},
    {"text": "Training for bodybuilding often involves a split routine, targeting different muscle groups on specific days to maximize growth and recovery."},
    {"text": "The golden era of bodybuilding, which spanned the 1960s and 1970s, featured legends like Arnold Schwarzenegger and Frank Zane, who emphasized symmetry and proportion over sheer size."},
    {"text": "Chris Bumstead has a large following on social media, where he shares motivational content, training tips, and behind-the-scenes glimpses of his bodybuilding journey."},
    {"text": "Mr. Olympia champions are often considered the pinnacle of bodybuilding excellence, representing the best in the sport across various categories."},
    {"text": "Bodybuilding competitions are highly competitive, requiring athletes to peak at the right time by manipulating training, diet, and even water intake."}
]

In [18]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
texts = [doc["text"] for doc in documents]
vectorstore = FAISS.from_texts(texts, embedding_model)

In [19]:
local_model = pipeline("text2text-generation", model="google/flan-t5-small")
llm = HuggingFacePipeline(pipeline=local_model)

In [20]:
retriever = vectorstore.as_retriever()

In [21]:
prompt_template = PromptTemplate(
    template="Context: {context}\n\nQuestion: {question}\n\nAnswer:",
    input_variables=["context", "question"]
)

In [22]:
combine_documents_chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt_template)

In [23]:
qa_chain = RetrievalQA(combine_documents_chain=combine_documents_chain, retriever=retriever)

In [25]:
!ngrok authtoken 'YOUR_NGROK_AUTHTOKEN'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [30]:
app = FastAPI()
nest_asyncio.apply()

class QuestionRequest(BaseModel):
    question: str

@app.post("/ask")
def ask_question(request: QuestionRequest):
    query = request.question
    response = qa_chain.run(query)
    return {"question": query, "answer": response}

In [33]:
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

import uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://cc35-34-172-97-72.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [742]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, i

INFO:     41.225.80.183:0 - "POST /ask HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


INFO:     41.225.80.183:0 - "POST /ask HTTP/1.1" 200 OK


...

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


INFO:     41.225.80.183:0 - "POST /ask HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


INFO:     41.225.80.183:0 - "POST /ask HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [742]


In [32]:
!pkill ngrok

In [11]:
query = "who is cbum?"
response = qa_chain.run(query)
print("Query:", query)
print("Response:", response)

<ipython-input-11-339b554f30c5>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Query: who is cbum?
Response: professional bodybuilder


In [13]:
query = "who is arnold?"
response = qa_chain.run(query)
print("Query:", query)
print("Response:", response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Query: who is arnold?
Response: a seven-time Mr. Olympia champion


In [14]:
query = "who is chris bumsted?"
response = qa_chain.run(query)
print("Query:", query)
print("Response:", response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Query: who is chris bumsted?
Response: professional bodybuilder
